# SAM 3D - 單張圖片轉完整 3D 模型

**使用 Tomalin18/Sam-3D 倉庫**

**使用說明**：
1. 點擊選單 **「執行階段 → 變更執行階段類型」**，選擇 **「T4 GPU」**
2. 依序執行每個 Cell（按 `Shift + Enter`）

---

## 步驟 1: 安裝 SAM 3D

In [ ]:
# 克隆 Sam-3D 倉庫
!git clone https://github.com/Tomalin18/Sam-3D.git
%cd Sam-3D

print("✅ Sam-3D 倉庫克隆完成！")

## 步驟 2: 安裝依賴

In [ ]:
%%capture
# 安裝 PyTorch (CUDA 版本)
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121

# 安裝其他依賴
!pip install huggingface-hub transformers diffusers accelerate
!pip install trimesh plyfile imageio opencv-python pillow
!pip install omegaconf hydra-core einops
!pip install rembg

print("✅ 依賴安裝完成！")

## 步驟 3: 登入 Hugging Face

In [ ]:
from huggingface_hub import login

# 輸入您的 Hugging Face Token
token = input("請輸入 Hugging Face Token: ")
login(token=token)

print("✅ 已成功登入 Hugging Face")

## 步驟 4: 修復環境變數問題

In [ ]:
import os

# 設定 CUDA_HOME 為 Colab 的標準路徑
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['CONDA_PREFIX'] = '/usr/local'

print("✅ 環境變數設定完成")

## 步驟 5: 下載模型權重

In [ ]:
from huggingface_hub import snapshot_download

# 下載 SAM 3D Objects 模型
print("📥 開始下載模型權重（約 2-3 分鐘）...")

model_path = snapshot_download(
    repo_id="facebook/sam-3d-objects",
    local_dir="checkpoints/hf",
    local_dir_use_symlinks=False
)

print(f"✅ 模型已下載到: {model_path}")

## 步驟 6: 上傳您的圖片

In [ ]:
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt

# 上傳圖片
print("📤 請選擇您要轉換的圖片（支援 JPG, PNG）...")
uploaded = files.upload()

# 取得檔案名稱
filename = list(uploaded.keys())[0]

# 預覽圖片
img = Image.open(filename)
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis('off')
plt.title(f'上傳的圖片: {filename}')
plt.show()

print(f"✅ 圖片尺寸: {img.size}")
print(f"✅ 已上傳: {filename}")

## 步驟 7: 載入 SAM 3D 模型

In [ ]:
import sys
sys.path.append('notebook')

from inference import Inference, load_image

# 載入模型
print("🔄 載入 SAM 3D 模型...")
config_path = "checkpoints/hf/pipeline.yaml"
inference = Inference(config_path, compile=False)

print("✅ 模型載入完成！")

## 步驟 8: 生成 3D 模型

In [ ]:
import time

# 載入圖片
image = load_image(filename)

# 開始生成
print("🎨 開始生成 3D 模型...")
start_time = time.time()

output = inference(image, seed=42)

elapsed_time = time.time() - start_time
print(f"✅ 3D 模型生成完成！耗時: {elapsed_time:.2f} 秒")

# 儲存為 .ply 格式 (Gaussian Splat)
output_filename = "output.ply"
output["gs"].save_ply(output_filename)

print(f"💾 已儲存: {output_filename}")

## 步驟 9: 下載 3D 模型

In [ ]:
from google.colab import files

# 下載 .ply 檔案
files.download(output_filename)

print(f"⬇️ 已下載: {output_filename}")
print("")
print("📌 如何查看 3D 模型：")
print("1. 線上查看器: https://playcanvas.com/supersplat/editor")
print("2. 您的網頁 UI: http://localhost:3000 (已有 3D 查看器)")
print("3. 桌面軟體: Blender, MeshLab")

---

## 🎉 完成！

您現在擁有一個完整的 360° 3D 模型！

### 可選：轉換為 OBJ 格式

In [ ]:
# 安裝 pymeshlab 用於格式轉換
!pip install -q pymeshlab

import pymeshlab as ml

# 載入 .ply 並轉換為 .obj
ms = ml.MeshSet()
ms.load_new_mesh('output.ply')
ms.save_current_mesh('output.obj')

# 下載 .obj 檔案
files.download('output.obj')
print("⬇️ 已下載: output.obj")